# Kubernetes Pods

## Useful functions

Here we defined some useful functions to be able to run this notebook in isolation:

In [1]:
#
# checkState TYPE NAME DESIRED
#
# description: Used to check state of kubernetes entity (useful to wait for conditions)
#
# example:
#    checkState pod k8s-demo Running
#
# returns:
#    0: If in desired state
#    1: If not
#
checkState() {
    TYPE=$1; shift
    NAME=$1; shift
    DESIRED=$1; shift
    
    QUIET="-q"
    [ ! -z "$1" ] && QUIET=""
    
    kubectl get $TYPE | grep $NAME | grep $QUIET $DESIRED && return 0
    
    return 1
}
# set -x; checkState pod k8s-demo Running ; echo $?

#
# checkPresent TYPE NAME DESIRED
#
# description: Used to check presence of kubernetes entity (useful to wait for conditions)
#
# example:
#    checkPresent pod k8s-demo
#
# returns:
#    0: If present
#    1: If not
#
checkPresent() {
    TYPE=$1; shift
    NAME=$1; shift
    
    QUIET="-q"
    [ ! -z "$1" ] && QUIET=""
    
    kubectl get $TYPE | grep $NAME && return 0
    
    return 1
}
# set -x; checkPresent pod k8s-demo ; echo $?

untilState() {
    TYPE=$1; shift
    NAME=$1; shift
    DESIRED=$1; shift
    
    QUIET="-q"
    [ ! -z "$1" ] && QUIET="-v"
    
    SLEEP=4
    while ! checkState $TYPE $NAME $DESIRED $QUIET; do
        sleep $SLEEP
    done
}
# set -x; untilstate pod k8s-demo Running

untilNotState() {
    TYPE=$1; shift
    NAME=$1; shift
    DESIRED=$1; shift
    
    QUIET="-q"
    [ ! -z "$1" ] && QUIET="-v"
    
    SLEEP=4
    while checkState $TYPE $NAME $DESIRED $QUIET; do
        sleep $SLEEP
    done
}
# set -x; untilNotState pod k8s-demo Terminating

untilPresent() {
    TYPE=$1; shift
    NAME=$1; shift
    
    QUIET="-q"
    [ ! -z "$1" ] && QUIET="-v"
    
    SLEEP=4
    while ! checkPresent $TYPE $NAME $QUIET; do
        sleep $SLEEP
    done
}
# set -x; untilPresent pod k8s-demo

untilNotPresent() {
    TYPE=$1; shift
    NAME=$1; shift
    
    QUIET="-q"
    [ ! -z "$1" ] && QUIET="-v"
    
    SLEEP=4
    while checkPresent $TYPE $NAME $QUIET; do
        sleep $SLEEP
    done
}
# set -x; untilNotPresent pod k8s-demo

#
# cont
#
# description: Used to continue notebook execution despite non-zero return code of previous command
#
cont() {
    return 0
}

set +x

In [2]:
kubectl get all

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deploy/k8s-demo   2         2         2            2           4m

NAME                     DESIRED   CURRENT   READY     AGE
rs/k8s-demo-6b76bd84d7   2         2         2         4m

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deploy/k8s-demo   2         2         2            2           4m

NAME                     DESIRED   CURRENT   READY     AGE
rs/k8s-demo-6b76bd84d7   2         2         2         4m

NAME                           READY     STATUS    RESTARTS   AGE
po/k8s-demo-6b76bd84d7-mzmkz   1/1       Running   0          4m
po/k8s-demo-6b76bd84d7-w8s8x   1/1       Running   0          4m

NAME             TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
svc/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   2d


## Cleanup

Let's first verify that we have no Pods, Deployments, Services running in the default namespace by running the '*kubectl get all' command

Then delete any running items (except '*svc/kubernetes*') if necessary

In [3]:
kubectl get all

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deploy/k8s-demo   2         2         2            2           4m

NAME                     DESIRED   CURRENT   READY     AGE
rs/k8s-demo-6b76bd84d7   2         2         2         4m

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deploy/k8s-demo   2         2         2            2           4m

NAME                     DESIRED   CURRENT   READY     AGE
rs/k8s-demo-6b76bd84d7   2         2         2         4m

NAME                           READY     STATUS    RESTARTS   AGE
po/k8s-demo-6b76bd84d7-mzmkz   1/1       Running   0          4m
po/k8s-demo-6b76bd84d7-w8s8x   1/1       Running   0          4m

NAME             TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
svc/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   2d


In [4]:
checkPresent pod    2-containers-in-a-pod &&
    kubectl delete po/2-containers-in-a-pod
checkPresent deploy k8s-demo              &&
    kubectl delete deploy k8s-demo
checkPresent svc    k8s-demo              &&
    kubectl delete service k8s-demo-service
cont

k8s-demo   2         2         2            2           4m
deployment "k8s-demo" deleted


In [5]:
kubectl get all

NAME             TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
svc/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   2d


We've seen already how we can interrogate our cluster server using the kubectl tool, obtaining information about objects such as some system pods, in the system.

Now let's create our own pods using kubectl.

Normally we would not create pods directly using command-line arguments, as we will do here.

We would create objects in a repeatable, declarative way from yaml files, but we will do that later.

The below ```kubectl run``` command will create 2 pods (replicas), each running just 1 container from the k8s-demo image.

Note that we can specify labels on any kubernetes objects, using the -l option followd by key=value pairs, e.g.
    -l "key1=val1,key2=val2"
    
These labels are extremely useful allowing us to select a group of objects using a label selector.

Labels might be used to mark components of the cluster as dev, test or production for example.

Or they might be used to signal hardware specifics of a node, to distinguish physical nodes with SSD disks, or GPU which could then be prioritized for particular types of processing which might be disk i/o or CPU intensive.

k8sdemo is a small Docker image of 6MBy on the Docker hub [HERE](https://hub.docker.com/r/mjbright/k8s-demo/tags/)

In [6]:
kubectl get all

NAME             TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
svc/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   2d


In [7]:
kubectl run k8s-demo -l "app=k8s-demo" --image=mjbright/k8s-demo:1 --replicas=2 --port=8080

deployment "k8s-demo" created


In [8]:
kubectl get all

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deploy/k8s-demo   2         2         2            0           1s

NAME                     DESIRED   CURRENT   READY     AGE
rs/k8s-demo-6b76bd84d7   2         2         0         1s

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deploy/k8s-demo   2         2         2            0           1s

NAME                     DESIRED   CURRENT   READY     AGE
rs/k8s-demo-6b76bd84d7   2         2         0         1s

NAME                           READY     STATUS              RESTARTS   AGE
po/k8s-demo-6b76bd84d7-c95nv   0/1       ContainerCreating   0          0s
po/k8s-demo-6b76bd84d7-kfrnb   0/1       ContainerCreating   0          1s

NAME             TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
svc/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   2d


In the above output we see that the only running resource initially was an internal service.

Once we perform '''kubectl run'' this spawns a **deployment** called *k8s-demo*, which creates a corrsponding **replicaset** which in turn spawns 2 **pods**.

As we performed the '''kubectl get all''' immediately after the run, we see the status before the pod containers have been created ... in fact there is a delay whilst the images are downloaded (for a first run) and pods instantiated.

Re-running '''kubectl get all''' shows the pods as *running* now.

In [9]:
untilState pod k8s-demo Running -q &&
    kubectl get all

k8s-demo-6b76bd84d7-c95nv   1/1       Running   0          4s
k8s-demo-6b76bd84d7-kfrnb   1/1       Running   0          5s
NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deploy/k8s-demo   2         2         2            2           5s

NAME                     DESIRED   CURRENT   READY     AGE
rs/k8s-demo-6b76bd84d7   2         2         2         5s

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deploy/k8s-demo   2         2         2            2           5s

NAME                     DESIRED   CURRENT   READY     AGE
rs/k8s-demo-6b76bd84d7   2         2         2         5s

NAME                           READY     STATUS    RESTARTS   AGE
po/k8s-demo-6b76bd84d7-c95nv   1/1       Running   0          4s
po/k8s-demo-6b76bd84d7-kfrnb   1/1       Running   0          5s

NAME             TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
svc/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   2d


... with more information, using option '*-o wide*'

In [10]:
kubectl get all -o wide

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE       CONTAINERS   IMAGES                SELECTOR
deploy/k8s-demo   2         2         2            2           6s        k8s-demo     mjbright/k8s-demo:1   app=k8s-demo

NAME                     DESIRED   CURRENT   READY     AGE       CONTAINERS   IMAGES                SELECTOR
rs/k8s-demo-6b76bd84d7   2         2         2         6s        k8s-demo     mjbright/k8s-demo:1   app=k8s-demo,pod-template-hash=2632684083

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE       CONTAINERS   IMAGES                SELECTOR
deploy/k8s-demo   2         2         2            2           6s        k8s-demo     mjbright/k8s-demo:1   app=k8s-demo

NAME                     DESIRED   CURRENT   READY     AGE       CONTAINERS   IMAGES                SELECTOR
rs/k8s-demo-6b76bd84d7   2         2         2         6s        k8s-demo     mjbright/k8s-demo:1   app=k8s-demo,pod-template-hash=2632684083

NAME              


Note in the above output we can see that our k8s-demo pods are running on kube-node-1 and kube-node-2.

## Creating resources using yaml specifications

### Obtaining yaml as output

Using the option '-o yaml' to any 'kubectl get' command we can obtain a yaml definition, so instead of

In [11]:
kubectl get deploy/k8s-demo -o wide

NAME       DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE       CONTAINERS   IMAGES                SELECTOR
k8s-demo   2         2         2            2           6s        k8s-demo     mjbright/k8s-demo:1   app=k8s-demo


we get ...

In [12]:
kubectl get deploy/k8s-demo -o yaml

apiVersion: extensions/v1beta1
kind: Deployment
metadata:
  annotations:
    deployment.kubernetes.io/revision: "1"
  creationTimestamp: 2018-03-12T10:47:37Z
  generation: 1
  labels:
    app: k8s-demo
  name: k8s-demo
  namespace: default
  resourceVersion: "289358"
  selfLink: /apis/extensions/v1beta1/namespaces/default/deployments/k8s-demo
  uid: c7c7c1cc-25e2-11e8-a061-02420ac00002
spec:
  replicas: 2
  selector:
    matchLabels:
      app: k8s-demo
  strategy:
    rollingUpdate:
      maxSurge: 1
      maxUnavailable: 1
    type: RollingUpdate
  template:
    metadata:
      creationTimestamp: null
      labels:
        app: k8s-demo
    spec:
      containers:
      - image: mjbright/k8s-demo:1
        imagePullPolicy: IfNotPresent
        name: k8s-demo
        ports:
        - containerPort: 8080
          protocol: TCP
        resources: {}
        terminationMessagePath: /dev/termination-log
        terminationMessagePolicy: File
      dnsPolicy: ClusterFirst
      restartPol

## Obtaining the example yaml

If you are using one of the VMS created for this lab then the example code will have been placed under:
~/src/git/ContainerOrchestration.Labs/Orchestration-Kubernetes/

If you are running in your own environment, or under play-with-kubernetes.com, you will first need to clone the repository:

```git clone https://github.com/ContainerOrchestration/Labs```

**NOTE**: You may also need to checkout the current branch, e.g.:FOSSAsia2018

```git checkout FOSSAsia2018```

## Inspecting the example

In the example yaml below, we see that we specify a Pod to be created (this is the smallest unit of execution in a Kubernetes cluster) which will contains 2 containers.

The yaml specifies which Kubernetes api version to use, the "*kind*" of object to be managed, and it's name.

In [13]:
cat examples/3.pod_example.yaml


apiVersion: v1
kind: Pod
metadata:
  name: 2-containers-in-a-pod
spec:
  containers:
  - name: k8s-demo
    image: mjbright/k8s-demo:1
    ports:
    - containerPort: 9876
  - name: shell
    image: alpine
    command:
      - "bin/sh"
      - "-c"
      - "while true; do date; sleep 1; done"



Let's first cleanup by verifying that this pod isn't already running.

We can use our yaml definition to delete any instances which may be running:

In [14]:
checkPresent pod 2-containers-in-a-pod && {
    echo; echo "-- Deleting pod:"
    kubectl delete -f examples/3.pod_example.yaml

    echo; echo "-- Waiting for pod to terminate:"
    untilNotPresent pod 2-containers-in-a-pod 
    }
cont

If we deleted any instances of "*2-containers-in-a-pod*" we need to wait for them to have terminated:


In [15]:
kubectl get all

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deploy/k8s-demo   2         2         2            2           8s

NAME                     DESIRED   CURRENT   READY     AGE
rs/k8s-demo-6b76bd84d7   2         2         2         8s

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deploy/k8s-demo   2         2         2            2           8s

NAME                     DESIRED   CURRENT   READY     AGE
rs/k8s-demo-6b76bd84d7   2         2         2         8s

NAME                           READY     STATUS    RESTARTS   AGE
po/k8s-demo-6b76bd84d7-c95nv   1/1       Running   0          7s
po/k8s-demo-6b76bd84d7-kfrnb   1/1       Running   0          8s

NAME             TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
svc/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   2d


Now we can create our pod as follows:

In [16]:
kubectl create -f examples/3.pod_example.yaml

pod "2-containers-in-a-pod" created


In [17]:
kubectl get po/2-containers-in-a-pod

NAME                    READY     STATUS              RESTARTS   AGE
2-containers-in-a-pod   0/2       ContainerCreating   0          0s


In [18]:
untilState pod 2-containers-in-a-pod Running &&
    kubectl get po/2-containers-in-a-pod

NAME                    READY     STATUS    RESTARTS   AGE
2-containers-in-a-pod   2/2       Running   0          5s


In [19]:
kubectl describe po/2-containers-in-a-pod

Name:         2-containers-in-a-pod
Namespace:    default
Node:         kube-node-2/10.192.0.4
Start Time:   Mon, 12 Mar 2018 10:47:46 +0000
Labels:       <none>
Annotations:  <none>
Status:       Running
IP:           10.192.3.19
Containers:
  k8s-demo:
    Container ID:   docker://b6b2b87540dc4d518da737d35363db050ee7d72c13546736a332f6cedaeca826
    Image:          mjbright/k8s-demo:1
    Image ID:       docker-pullable://mjbright/k8s-demo@sha256:6d84d697796e0a529028808ba76e92482c4ca0c408e9f9eae8aba6f23c173264
    Port:           9876/TCP
    State:          Running
      Started:      Mon, 12 Mar 2018 10:47:48 +0000
    Ready:          True
    Restart Count:  0
    Environment:    <none>
    Mounts:
      /var/run/secrets/kubernetes.io/serviceaccount from default-token-xdvkn (ro)
  shell:
    Container ID:  docker://7210f7127f8eee2f8a4a80d6c2f5ae3f43c2d5f78b862f5f30d77d55d3d71f7e
    Image:         alpine
    Image ID:      docker-pullable://alpine@sha256:7b848083f93822dd21b0a2f14a1

In [20]:
kubectl exec -it 2-containers-in-a-pod -n default echo hello world

Defaulting container name to k8s-demo.
Use 'kubectl describe pod/2-containers-in-a-pod -n default' to see all of the containers in this pod.
hello world


In [21]:
kubectl logs 2-containers-in-a-pod shell | head -3

Mon Mar 12 10:47:49 UTC 2018
Mon Mar 12 10:47:50 UTC 2018
Mon Mar 12 10:47:51 UTC 2018


In [22]:
kubectl delete po/2-containers-in-a-pod

pod "2-containers-in-a-pod" deleted


In [23]:
kubectl get pods

NAME                        READY     STATUS        RESTARTS   AGE
2-containers-in-a-pod       2/2       Terminating   0          8s
k8s-demo-6b76bd84d7-c95nv   1/1       Running       0          16s
k8s-demo-6b76bd84d7-kfrnb   1/1       Running       0          17s


## What happens if a pod dies?

In [24]:
kubectl get pods -o wide

NAME                        READY     STATUS        RESTARTS   AGE       IP            NODE
2-containers-in-a-pod       2/2       Terminating   0          8s        10.192.3.19   kube-node-2
k8s-demo-6b76bd84d7-c95nv   1/1       Running       0          16s       10.192.2.13   kube-node-1
k8s-demo-6b76bd84d7-kfrnb   1/1       Running       0          17s       10.192.3.18   kube-node-2


When we deleted the 2-containers-in-a-pod Pod it just died.

If we delete one of our k8s-demo Pods we will see that a new Pod is created automatically, this is because it is managed by a ReplicaSet.

Running a Pod via "kubectl run" creates a Deployment which then creates a ReplicaSet.

Let's kill one of our Pods and see what happens


In [25]:
kubectl get pod | grep k8s-demo | head -1 | awk '{ print $1;}'

POD1=$(kubectl get pod | grep k8s-demo | head -1 | awk '{ print $1;}')

k8s-demo-6b76bd84d7-c95nv


In [26]:
kubectl delete pod $POD1

pod "k8s-demo-6b76bd84d7-c95nv" deleted


In [27]:
kubectl get pods -o wide

NAME                        READY     STATUS              RESTARTS   AGE       IP            NODE
2-containers-in-a-pod       2/2       Terminating         0          10s       10.192.3.19   kube-node-2
k8s-demo-6b76bd84d7-c95nv   1/1       Terminating         0          18s       10.192.2.13   kube-node-1
k8s-demo-6b76bd84d7-kfrnb   1/1       Running             0          19s       10.192.3.18   kube-node-2
k8s-demo-6b76bd84d7-qcml8   0/1       ContainerCreating   0          0s        <none>        kube-node-1


In [28]:
sleep 10; kubectl get pods -o wide

NAME                        READY     STATUS        RESTARTS   AGE       IP            NODE
2-containers-in-a-pod       2/2       Terminating   0          20s       10.192.3.19   kube-node-2
k8s-demo-6b76bd84d7-kfrnb   1/1       Running       0          29s       10.192.3.18   kube-node-2
k8s-demo-6b76bd84d7-qcml8   1/1       Running       0          10s       10.192.2.14   kube-node-1


We see that our Pod was deleted but that a new one was created, this is what we expect when our Pod is managed by a ReplicaSet.

## What happens if a node dies?
We should see similar behaviour is a Node dies.

Anyone want to try this?

# Scaling our application

As our Pods are managed by a **ReplicaSet** we can scale the number of replicas by using the scale command on the **Deployment**:

In [29]:
kubectl scale --replicas=3 deploy/k8s-demo

deployment "k8s-demo" scaled


In [30]:
kubectl get all

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deploy/k8s-demo   3         2         2            2           32s

NAME                     DESIRED   CURRENT   READY     AGE
rs/k8s-demo-6b76bd84d7   3         2         2         32s

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deploy/k8s-demo   3         3         3            2           32s

NAME                     DESIRED   CURRENT   READY     AGE
rs/k8s-demo-6b76bd84d7   3         3         2         32s

NAME                           READY     STATUS              RESTARTS   AGE
po/2-containers-in-a-pod       2/2       Terminating         0          23s
po/k8s-demo-6b76bd84d7-f2x7c   0/1       ContainerCreating   0          0s
po/k8s-demo-6b76bd84d7-kfrnb   1/1       Running             0          32s
po/k8s-demo-6b76bd84d7-qcml8   1/1       Running             0          13s

NAME             TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
svc/kubernetes   ClusterIP   10.96.0.1    

In [31]:
kubectl get pods -o wide

NAME                        READY     STATUS              RESTARTS   AGE       IP            NODE
2-containers-in-a-pod       2/2       Terminating         0          24s       10.192.3.19   kube-node-2
k8s-demo-6b76bd84d7-f2x7c   0/1       ContainerCreating   0          1s        <none>        kube-node-1
k8s-demo-6b76bd84d7-kfrnb   1/1       Running             0          33s       10.192.3.18   kube-node-2
k8s-demo-6b76bd84d7-qcml8   1/1       Running             0          14s       10.192.2.14   kube-node-1


### Scaling down

We can of course scale back down:

In [32]:
kubectl scale --replicas=2 deploy/k8s-demo

deployment "k8s-demo" scaled


In [33]:
untilNotState pod k8s-demo Terminating

k8s-demo-6b76bd84d7-f2x7c   0/1       Terminating   0          3s
k8s-demo-6b76bd84d7-f2x7c   1/1       Terminating   0          6s
k8s-demo-6b76bd84d7-f2x7c   1/1       Terminating   0          8s
k8s-demo-6b76bd84d7-f2x7c   1/1       Terminating   0          11s
k8s-demo-6b76bd84d7-f2x7c   1/1       Terminating   0          14s
k8s-demo-6b76bd84d7-f2x7c   1/1       Terminating   0          17s
k8s-demo-6b76bd84d7-f2x7c   1/1       Terminating   0          20s
k8s-demo-6b76bd84d7-f2x7c   1/1       Terminating   0          22s
k8s-demo-6b76bd84d7-f2x7c   1/1       Terminating   0          25s
k8s-demo-6b76bd84d7-f2x7c   1/1       Terminating   0          28s
k8s-demo-6b76bd84d7-f2x7c   1/1       Terminating   0          31s
k8s-demo-6b76bd84d7-f2x7c   1/1       Terminating   0          33s
k8s-demo-6b76bd84d7-f2x7c   1/1       Terminating   0          36s
k8s-demo-6b76bd84d7-f2x7c   1/1       Terminating   0          39s
k8s-demo-6b76bd84d7-f2x7c   1/1       Terminating   0          42

In [34]:
kubectl get pods -o wide

NAME                        READY     STATUS        RESTARTS   AGE       IP            NODE
2-containers-in-a-pod       0/2       Terminating   0          1m        10.192.3.19   kube-node-2
k8s-demo-6b76bd84d7-kfrnb   1/1       Running       0          1m        10.192.3.18   kube-node-2
k8s-demo-6b76bd84d7-qcml8   1/1       Running       0          1m        10.192.2.14   kube-node-1


## Accessing our application

To access our application pods we should expose them as a service.
This would have the advantage of being able to address a virtual ip address for our service as pods themselves come and go (so we don't need to determine their address), and also allows load balancing between pods.

### Accessing our application - directly accessing Pods

But there are other more basic ways that we will investigate first.

If we perform '''kubectl get pods -o wide''' we will see the cluster ip of our pods, enabling us to access directly to the pods.

In [41]:
kubectl get pods -o wide 

NAME                        READY     STATUS    RESTARTS   AGE       IP            NODE
k8s-demo-6b76bd84d7-kfrnb   1/1       Running   0          5m        10.192.3.18   kube-node-2
k8s-demo-6b76bd84d7-qcml8   1/1       Running   0          5m        10.192.2.14   kube-node-1


In [42]:
POD1=$(kubectl get pod -o wide --no-headers | head -1 | awk "/k8s-demo/ {print \$1;}")
POD2=$(kubectl get pod -o wide --no-headers | tail -1 | awk "/k8s-demo/ {print \$1;}")
POD1_IP=$(kubectl get pod -o wide --no-headers | head -1 | awk "/k8s-demo/ {print \$6;}")
POD2_IP=$(kubectl get pod -o wide --no-headers | tail -1 | awk "/k8s-demo/ {print \$6;}")

echo POD1=$POD1 POD1_IP=$POD1_IP
echo POD2=$POD2 POD2_IP=$POD2_IP

POD1=k8s-demo-6b76bd84d7-kfrnb POD1_IP=10.192.3.18
POD2=k8s-demo-6b76bd84d7-qcml8 POD2_IP=10.192.2.14


Let's look at the yaml information for POD1:

In [43]:
kubectl get pod $POD1 -o yaml

apiVersion: v1
kind: Pod
metadata:
  creationTimestamp: 2018-03-12T10:47:37Z
  generateName: k8s-demo-6b76bd84d7-
  labels:
    app: k8s-demo
    pod-template-hash: "2632684083"
  name: k8s-demo-6b76bd84d7-kfrnb
  namespace: default
  ownerReferences:
  - apiVersion: extensions/v1beta1
    blockOwnerDeletion: true
    controller: true
    kind: ReplicaSet
    name: k8s-demo-6b76bd84d7
    uid: c7c8e078-25e2-11e8-a061-02420ac00002
  resourceVersion: "289356"
  selfLink: /api/v1/namespaces/default/pods/k8s-demo-6b76bd84d7-kfrnb
  uid: c7cb73e1-25e2-11e8-a061-02420ac00002
spec:
  containers:
  - image: mjbright/k8s-demo:1
    imagePullPolicy: IfNotPresent
    name: k8s-demo
    ports:
    - containerPort: 8080
      protocol: TCP
    resources: {}
    terminationMessagePath: /dev/termination-log
    terminationMessagePolicy: File
    volumeMounts:
    - mountPath: /var/run/secrets/kubernetes.io/serviceaccount
      name: default-token-xdvkn
      readOnly: true
  dnsPolicy: ClusterFirst
 

We can see that the containerPort 8080/TCP is exposed (as we specified on our initial '*kubectl run*' command).

So now we can access to the pods directly on their port 8080.

In [44]:
curl http://$POD1_IP:8080




                               
                    (((((((((                  
               .(((((((((((((((((.             
           .((((((((((((&((((((((((((.         
       /((((((((((((((((@((((((((((((((((/     
      ((((((((((((((((((@((((((((((((((((((    
     *(((((##((((((@@@@@@@@@@@((((((%#(((((*   
     (((((((@@@(@@@@#((@@@((#@@@@(@@@(((((((   
    *(((((((((@@@@(((((@@@(((((@@@@(((((((((,  
    (((((((((@@@%@@@@((@@@((@@@@%@@@(((((((((  
   .(((((((((@@((((@@@@@@@@@@@((((@@(((((((((. 
   (((((((((&@@(((((@@@(((@@@(((((@@&((((((((( 
   (((((((((&@@@@@@@@@@#(#@@@@@@@@@@&((((((((( 
  ((((((@@@@@@@@(((((@@@@@@@(((((&@@@@@@@((((((
  (((((((((((%@@((((%@@@(@@@%((((@@&(((((((((((
   ((((((((((((@@@((@@%(((%@@((@@@(((((((((((( 
     (((((((((((#@@@@%(((((&@@@@#(((((((((((   
      /(((((((((((@@@@@@@@@@@@@(((((((((((/    
        (((((((((@@(((((((((((@@(((((((((      
          (((((((&(((((((((((((&(((((((        
           /(((((((((((((((((((((((((/         
     

In [45]:
curl http://$POD2_IP:8080




                               
                    (((((((((                  
               .(((((((((((((((((.             
           .((((((((((((&((((((((((((.         
       /((((((((((((((((@((((((((((((((((/     
      ((((((((((((((((((@((((((((((((((((((    
     *(((((##((((((@@@@@@@@@@@((((((%#(((((*   
     (((((((@@@(@@@@#((@@@((#@@@@(@@@(((((((   
    *(((((((((@@@@(((((@@@(((((@@@@(((((((((,  
    (((((((((@@@%@@@@((@@@((@@@@%@@@(((((((((  
   .(((((((((@@((((@@@@@@@@@@@((((@@(((((((((. 
   (((((((((&@@(((((@@@(((@@@(((((@@&((((((((( 
   (((((((((&@@@@@@@@@@#(#@@@@@@@@@@&((((((((( 
  ((((((@@@@@@@@(((((@@@@@@@(((((&@@@@@@@((((((
  (((((((((((%@@((((%@@@(@@@%((((@@&(((((((((((
   ((((((((((((@@@((@@%(((%@@((@@@(((((((((((( 
     (((((((((((#@@@@%(((((&@@@@#(((((((((((   
      /(((((((((((@@@@@@@@@@@@@(((((((((((/    
        (((((((((@@(((((((((((@@(((((((((      
          (((((((&(((((((((((((&(((((((        
           /(((((((((((((((((((((((((/         
     

**Note:** In a real deployment of course the actual Pod ips would not be externally accessible so we would want some way of making the nodes available on a well known address.